In [ ]:
!pip install d2l==0.16.

In [12]:
import torch
import torchvision
from d2l import torch as d2l
import torch.nn as nn

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')
#
# %cd /content/drive/My Drive
# !ls
# %cd d2l-dataset/

Mounted at /content/drive
/content/drive/My Drive
'Colab Notebooks'   d2l-dataset   d2l-zh   data   test.ipynb
/content/drive/My Drive/d2l-dataset


In [ ]:
all_images = torchvision.datasets.CIFAR10(train=True, root="../data", download=True)
d2l.show_images([all_images[i][0] for i in range(32)], 4, 8, scale=0.8);

In [6]:
train_augs = torchvision.transforms.Compose([
                      torchvision.transforms.RandomHorizontalFlip(),
                      torchvision.transforms.ToTensor()])
test_augs =  torchvision.transforms.Compose([
                      torchvision.transforms.ToTensor()])

In [7]:
def load_CIFAR10(is_train,augs,batch_size):
  dataset = torchvision.datasets.CIFAR10(root="../data", train=is_train,transform=augs, download=True)
  dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                    shuffle=is_train, num_workers=2)
  return dataloader

In [17]:
def train_batch(net, X, y, loss, trainer, devices):
    """用多GPU进行小批量训练"""
    if isinstance(X, list):
        # 微调BERT中所需（稍后讨论）
        X = [x.to(devices[0]) for x in X]
    else:
        X = X.to(devices[0])
    y = y.to(devices[0])
    net.train()
    trainer.zero_grad()
    pred = net(X)
    l = loss(pred, y)
    l.sum().backward()
    trainer.step()
    train_loss_sum = l.sum()
    train_acc_sum = d2l.accuracy(pred, y)
    return train_loss_sum, train_acc_sum

In [18]:
def train(net, train_iter, test_iter, loss, trainer, num_epochs,
               devices=d2l.try_all_gpus()):
    """用多GPU进行模型训练"""
    timer, num_batches = d2l.Timer(), len(train_iter)
    animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs], ylim=[0, 1],
                            legend=['train loss', 'train acc', 'test acc'])
    net = nn.DataParallel(net, device_ids=devices).to(devices[0])
    for epoch in range(num_epochs):
        # 4个维度：储存训练损失，训练准确度，实例数，特点数
        metric = d2l.Accumulator(4)
        for i, (features, labels) in enumerate(train_iter):
            timer.start()
            l, acc = train_batch(
                net, features, labels, loss, trainer, devices)
            metric.add(l, acc, labels.shape[0], labels.numel())
            timer.stop()
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                animator.add(epoch + (i + 1) / num_batches,
                             (metric[0] / metric[2], metric[1] / metric[3],
                              None))
        test_acc = d2l.evaluate_accuracy_gpu(net, test_iter)
        animator.add(epoch + 1, (None, None, test_acc))
    print(f'loss {metric[0] / metric[2]:.3f}, train acc '
          f'{metric[1] / metric[3]:.3f}, test acc {test_acc:.3f}')
    print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec on '
          f'{str(devices)}')


In [19]:
batch_size,devices,net = 256,d2l.try_all_gpus(),d2l.resnet18(10,3)

In [20]:
def init_weights(m):
  if type(m) in [nn.Linear,nn.Conv2d]:
    nn.init.xavier_uniform_(m.weight)

net.apply(init_weights)
def train_with_data_aug(train_augs, test_augs, net, lr=0.001):
    train_iter = load_CIFAR10(True, train_augs, batch_size)
    test_iter = load_CIFAR10(False, test_augs, batch_size)
    loss = nn.CrossEntropyLoss(reduction="none")
    trainer = torch.optim.Adam(net.parameters(), lr=lr)
    train(net, train_iter, test_iter, loss, trainer, 10, devices)


In [ ]:
train_with_data_aug(train_augs,test_augs,net)

Files already downloaded and verified
Files already downloaded and verified
